In [ ]:
from manim import *

config.media_width = "100%"
config.verbosity = "WARNING"
config.quality = "medium_quality"
config.preview = False

In [31]:
%%manim TaylorSeriesScene

class TaylorSeriesScene(Scene):
    def construct(self):
        axes = Axes(
            x_range=[-3, 3], 
            y_range=[-2, 3], 
            axis_config={"include_numbers": True}
        )
        axis_labels = axes.get_axis_labels(x_label="x", y_label="y")

        function_curve = axes.plot(lambda x: 1/(1+x**2), color=BLUE, stroke_width=4)
        
        singularity_barriers = VGroup(*[
            DashedLine(axes.c2p(x, -2), axes.c2p(x, 3), color=RED, stroke_opacity=0.5) 
            for x in [-1, 1]
        ])
        
        function_label = MathTex(r"f(x) = \frac{1}{1+x^2}", color=BLUE).to_corner(UL)

        self.play(Create(axes), Write(axis_labels))
        self.play(Create(function_curve), Create(singularity_barriers), Write(function_label))

        def taylor_expansion(x, n):
            return sum((-1)**i * x**(2*i) for i in range(n + 1))

        taylor_curve = axes.plot(lambda x: taylor_expansion(x, 0), color=YELLOW, x_range=[-1.45, 1.45])
        order_label = MathTex("n=0", color=YELLOW).to_corner(UR)

        self.play(Create(taylor_curve), Write(order_label))
        self.wait()

        for order in range(1, 6):
            new_taylor_curve = axes.plot(
                lambda x: taylor_expansion(x, order), 
                color=YELLOW, 
                x_range=[-1.45, 1.45]
            )
            new_order_label = MathTex(f"n={order}", color=YELLOW).to_corner(UR)

            self.play(
                Transform(taylor_curve, new_taylor_curve),
                Transform(order_label, new_order_label),
                run_time=0.8
            )
            self.wait(0.2)
            
        self.wait(2)

Manim Community v0.19.1

In [32]:
%%manim ComplexSurfaceScene

class ComplexSurfaceScene(ThreeDScene):
    def construct(self):
        axes = ThreeDAxes(
            x_range=[-3, 3], y_range=[-3, 3], z_range=[0, 4],
            x_length=8, y_length=8, z_length=5,
            axis_config={"include_tip": True, "include_ticks": True, "color": GREY}
        )
        
        axes.add_coordinates(
            dict(zip([x for x in range(-3, 4) if x != 0], [x for x in range(-3, 4) if x != 0])),
            {}, 
            {}
        )

        imaginary_axis_labels = VGroup()
        imaginary_label_positions = [(1, "i"), (-1, "-i"), (2, "2i"), (-2, "-2i")]
        
        for position, label_text in imaginary_label_positions:
            label = MathTex(label_text, color=WHITE).scale(0.7)
            label.move_to(axes.c2p(0.4, position, 0))
            label.rotate(90*DEGREES, RIGHT).rotate(90*DEGREES, OUT)
            imaginary_axis_labels.add(label)

        real_curve_2d = axes.plot_parametric_curve(
            lambda x: np.array([x, 0, 1/(1+x**2)]),
            t_range=[-3, 3], color=BLUE, stroke_width=4
        )
        
        self.set_camera_orientation(phi=90*DEGREES, theta=-90*DEGREES)
        self.add(axes, real_curve_2d, imaginary_axis_labels)
        self.wait(1)

        self.move_camera(phi=60*DEGREES, theta=-45*DEGREES, run_time=2)

        max_height = 4.0
        magnitude_surface = Surface(
            lambda u, v: axes.c2p(u, v, min(max_height, 1 / (abs(1 + complex(u, v)**2) + 1e-6))),
            u_range=[-2.5, 2.5], v_range=[-2.5, 2.5],
            resolution=(28, 28),
            fill_color=BLUE, fill_opacity=0.3,
            checkerboard_colors=[BLUE, BLUE], 
            stroke_width=0.2, stroke_color=BLUE_A
        )

        self.play(FadeIn(magnitude_surface), run_time=1.5)

        pole_indicators = VGroup()
        for imaginary_coord in [-1, 1]:
            pole_indicators.add(DashedLine(
                start=axes.c2p(0, imaginary_coord, max_height), 
                end=axes.c2p(0, imaginary_coord, 0),     
                color=RED
            ))

        convergence_circle = Circle(radius=axes.get_x_unit_size(), color=YELLOW)
        convergence_circle.rotate(0, axis=RIGHT).move_to(axes.c2p(0,0,0))

        self.play(Create(pole_indicators), run_time=1.5)
        self.play(Create(convergence_circle), run_time=1.5)
        
        self.begin_ambient_camera_rotation(rate=0.1)
        self.wait(4)

Manim Community v0.19.1

In [33]:
%%manim ConvergenceRadiusScene

class ConvergenceRadiusScene(Scene):
    def construct(self):
        self.camera.background_color = BLACK
        
        complex_plane = ComplexPlane(
            x_range=[-3, 6, 1], y_range=[-3, 3, 1],
            background_line_style={"stroke_opacity": 0.2, "stroke_color": TEAL},
            axis_config={"color": GREY}
        )
        
        complex_plane.x_axis.add_numbers(
            x_values=[x for x in range(-2, 7) if x != 0], 
            font_size=24
        )
        
        positive_imaginary_label = MathTex("i", color=WHITE, font_size=24).next_to(complex_plane.n2p(1j), LEFT, buff=0.1)
        negative_imaginary_label = MathTex("-i", color=WHITE, font_size=24).next_to(complex_plane.n2p(-1j), LEFT, buff=0.1)
        
        positive_pole = Dot(complex_plane.n2p(1j), color=RED)
        negative_pole = Dot(complex_plane.n2p(-1j), color=RED)
        
        self.add(complex_plane, positive_imaginary_label, negative_imaginary_label, positive_pole, negative_pole)

        center_position = ValueTracker(0)

        center_dot = always_redraw(lambda: 
            Dot(complex_plane.n2p(center_position.get_value()), color=YELLOW)
        )
        
        center_trace = TracedPath(center_dot.get_center, stroke_color=WHITE, stroke_opacity=0.4, stroke_width=6)
        self.add(center_trace)

        radius_line_to_pole = always_redraw(lambda: 
            Line(
                start=complex_plane.n2p(center_position.get_value()), 
                end=complex_plane.n2p(1j), 
                color=ORANGE, 
                stroke_opacity=0.8
            )
        )
        
        convergence_circle = always_redraw(lambda: 
            Circle(
                radius=np.sqrt(center_position.get_value()**2 + 1) * complex_plane.get_x_unit_size(),
                color=YELLOW, 
                fill_opacity=0.1, 
                stroke_width=2
            ).move_to(complex_plane.n2p(center_position.get_value()))
        )

        center_position_label = always_redraw(lambda: 
            MathTex(
                f"x_0 = {center_position.get_value():.2f}", 
                color=YELLOW
            ).to_corner(UL)
        )
        
        radius_formula_label = always_redraw(lambda: 
            MathTex(
                r"R = \sqrt{x_0^2 + 1} \approx " + f"{np.sqrt(center_position.get_value()**2 + 1):.2f}",
                color=ORANGE
            ).to_corner(UR)
        )

        self.play(
            FadeIn(convergence_circle), 
            FadeIn(center_dot), 
            Create(radius_line_to_pole), 
            Write(center_position_label), 
            Write(radius_formula_label)
        )
        self.wait(0.5)
        
        self.play(
            center_position.animate.set_value(4), 
            run_time=4, 
            rate_func=smooth
        )
        self.wait(0.5)
        
        self.play(
            center_position.animate.set_value(-1.5), 
            run_time=3, 
            rate_func=smooth
        )
        
        self.wait(2)

Manim Community v0.19.1